In [2]:
from google.colab import drive
drive.mount('/content/drive')
SAVE_DIR = '/content/drive/MyDrive/Colab_Notebooks2/LLM_files'

Mounted at /content/drive


In [ ]:
# Google Colab Setup Script for LLM + FA Detection EDA Flow

# 1. System update & dependencies
!apt-get update -qq && apt-get install -y build-essential clang bison flex libreadline-dev \
  gawk tcl-dev libffi-dev git mercurial graphviz xdot pkg-config python3 python3-pip \
  libboost-system-dev libboost-python-dev libboost-filesystem-dev zlib1g-dev

# 2. Clone and build Yosys
!git clone https://github.com/YosysHQ/yosys.git
%cd yosys
!git submodule update --init --recursive
!make -j$(nproc)
!make install
!cp -r /content/yosys SAVE_DIR



In [ ]:
!cp -r /content/yosys /content/drive/MyDrive/Colab_Notebooks2/LLM_files/

In [149]:
# 3. Go back and create project directories
import os
%cd /
verilog_path = os.path.join(SAVE_DIR, 'llm_fa/verilog_src')
!mkdir -p /content/llm_fa/verilog_src /content/llm_fa/scripts /content/llm_fa/stats /content/llm_fa/llm

# 4. Create example FA Verilog
fa_verilog = '''
module mystery_unit #(parameter WIDTH = 1) (
    input  wire [WIDTH-1:0] i_alpha,
    input  wire [WIDTH-1:0] i_beta,
    input  wire             i_gamma,
    output wire [WIDTH-1:0] o_sigma,
    output wire             o_delta
);

    wire [WIDTH-1:0] lvl1 [0:3];
    wire [WIDTH-1:0] junk [0:2];
    wire carry_internal;

    genvar idx;
    generate
        for (idx = 0; idx < WIDTH; idx = idx + 1) begin : loop_main

            assign junk[0][idx] = i_alpha[idx] | i_beta[idx];
            assign junk[1][idx] = junk[0][idx] & i_gamma;
            assign junk[2][idx] = ~(junk[1][idx] ^ i_gamma);
            assign lvl1[0][idx] = i_alpha[idx] ^ i_beta[idx];
            assign lvl1[1][idx] = lvl1[0][idx] ^ i_gamma;
            assign lvl1[2][idx] = ~(~(i_alpha[idx] & i_beta[idx]) & ~(i_beta[idx] & i_gamma));
            assign lvl1[3][idx] = ~(~(i_alpha[idx] & i_gamma) & ~lvl1[2][idx]);

            assign o_sigma[idx] = lvl1[1][idx];
        end
    endgenerate

    assign carry_internal = lvl1[3][0];
    assign o_delta = carry_internal;

endmodule
'''

with open("/content/llm_fa/verilog_src/fa_example_hard.v", "w") as f:
    f.write(fa_verilog)

# 5. Create FA cell primitive
fa_cell = '''
module top(input a, b, cin, output sum, output cout);
    FA_CELL fa (
        .a(a),
        .b(b),
        .cin(cin),
        .sum(sum),
        .cout(cout)
    );
endmodule

module FA_CELL(
    input  wire a,
    input  wire b,
    input  wire cin,
    output wire sum,
    output wire cout
);
  assign sum  = a ^ b ^ cin;
  assign cout = (a & b) | (cin & (a ^ b));
endmodule
'''
with open("/content/llm_fa/verilog_src/fa_cell.v", "w") as f:
    f.write(fa_cell)

# 6. Create yosys script to generate AIG
synth_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example_hard.v
hierarchy -auto-top
proc; opt; techmap
aigmap
write_aiger -ascii -symbols /content/llm_fa/design.aag
'''
#write_aiger /content/llm_fa/design.aig
with open("/content/llm_fa/scripts/synth.ys", "w") as f:
    f.write(synth_script)

print("✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys")

/
✅ Setup complete. You can now run Yosys on /content/llm_fa/scripts/synth.ys


In [154]:
# 7. Run Yosys synthesis to generate AIG
%cd /content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
!chmod +x yosys
!./yosys -V
!./yosys -q -s /content/llm_fa/scripts/synth.ys


/content/drive/MyDrive/Colab_Notebooks2/LLM_files/yosys
Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)


In [118]:
# 8-1. Generate LLM prompt to locate FA logic line-by-line in Verilog

prompt_template = f"""

Here is the Verilog source:

{fa_verilog}

For each FA instance in the code:
	•	Add a line-by-line comment at the end of each line that participates in a Full Adder logic, using this format:
wire ab = a & b; // FA #1
wire bc = b & cin; // FA #1
wire ac = a & cin; // FA #1
wire carry = ab | bc | ac; // FA #1
wire sum = a ^ b ^ cin; // FA #1
	•	If a second Full Adder exists in the code, label it as // FA #2, and so on.
	•	Do not change or delete any code.
	•	Leave unrelated lines untouched (no comments).
Please output a complete and compilable Verilog module.

Your output should begin with module and end with endmodule,Please do not include any Markdown code fences like ```verilog in your output. Output only raw Verilog code suitable for saving as a .v file.
"""
with open("/content/llm_fa/llm/llm_prompt_step1.txt", "w") as f:
    f.write(prompt_template)
print("🧠 LLM prompt for step1 generated. Ready to pass to a model like GPT for annotation.")



🧠 LLM prompt for step1 generated. Ready to pass to a model like GPT for annotation.


In [120]:
# 8-2. Call LLM to annotate Verilog code
import os
from openai import OpenAI

# Set your API key
os.environ["OPENAI_API_KEY"] = "USER-API-KEY"
client = OpenAI()

with open("/content/llm_fa/llm/llm_prompt_step1.txt", "r") as f:
    prompt = f.read()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
llm_result_step1 = response.choices[0].message.content
with open("/content/llm_fa/llm/llm_result_step1.txt", "w") as f:
    f.write(llm_result_step1)

print("📝 LLM result saved to llm_result_step1.txt")

module mystery_unit #(parameter WIDTH = 1) (
    input  wire [WIDTH-1:0] i_alpha,
    input  wire [WIDTH-1:0] i_beta,
    input  wire             i_gamma,
    output wire [WIDTH-1:0] o_sigma,
    output wire             o_delta
);

    wire [WIDTH-1:0] lvl1 [0:3];
    wire [WIDTH-1:0] junk [0:2];
    wire carry_internal;

    genvar idx;
    generate
        for (idx = 0; idx < WIDTH; idx = idx + 1) begin : loop_main

            assign junk[0][idx] = i_alpha[idx] | i_beta[idx];
            assign junk[1][idx] = junk[0][idx] & i_gamma;
            assign junk[2][idx] = ~(junk[1][idx] ^ i_gamma);
            assign lvl1[0][idx] = i_alpha[idx] ^ i_beta[idx]; // FA #1
            assign lvl1[1][idx] = lvl1[0][idx] ^ i_gamma; // FA #1
            assign lvl1[2][idx] = ~(~(i_alpha[idx] & i_beta[idx]) & ~(i_beta[idx] & i_gamma)); // FA #1
            assign lvl1[3][idx] = ~(~(i_alpha[idx] & i_gamma) & ~lvl1[2][idx]); // FA #1

            assign o_sigma[idx] = lvl1[1][idx]; // FA #1 
       

In [130]:
# 9-1. Convert FA logic to FA cell in Verilog
prompt_template = f"""
You are given a Verilog source code where lines involved in Full Adder logic are marked line-by-line with comments like:
{llm_result_step1}

Your task is to:
1. Detect each group of lines associated with a unique Full Adder based on the // FA #N comments.
2. For each FA group:
	•	Remove all the lines marked with // FA #N
	•	Replace them with a single instantiation of the module fa_cell, using the following format:
FA_CELL fa_N (
  .a(a), .b(b), .cin(cin),
  .sum(sum), .cout(carry)
);
	•	Important: The instance name faN must use the same number N from the comment (e.g., FA #1 → fa1, FA #2 → fa2, etc.)
3. Preserve all unrelated code, including unmarked lines, always keeping the surrounding structure intact.
4. Your output should begin with module and end with endmodule,Please do not include any Markdown code fences like ```verilog in your output. Output only raw Verilog code suitable for saving as a .v file.
"""

with open("/content/llm_fa/llm/llm_prompt_step2.txt", "w") as f:
    f.write(prompt_template)
print("🧠 LLM prompt for step2 generated. Ready to pass to a model like GPT for annotation.")


🧠 LLM prompt for step2 generated. Ready to pass to a model like GPT for annotation.


In [131]:
# 9-2. Call LLM to refactor FA logic to FA cell
import os
from openai import OpenAI

# Set your API key
os.environ["OPENAI_API_KEY"] = "USER-API-KEY"
client = OpenAI()
# 📝 2. Define Verilog with obfuscation (hard to identify FA)

with open("/content/llm_fa/llm/llm_prompt_step2.txt", "r") as f:
    prompt = f.read()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)
llm_result_step2 = response.choices[0].message.content
with open("/content/llm_fa/llm/llm_result_step2.txt", "w") as f:
    f.write(llm_result_step2)

with open("/content/llm_fa/verilog_src/fa_example_mod.v", "w") as f:
    f.write(llm_result_step2)


print("📝 LLM result saved to llm_result_step2.txt")

module mystery_unit #(parameter WIDTH = 1) (
    input  wire [WIDTH-1:0] i_alpha,
    input  wire [WIDTH-1:0] i_beta,
    input  wire             i_gamma,
    output wire [WIDTH-1:0] o_sigma,
    output wire             o_delta
);

    wire [WIDTH-1:0] lvl1 [0:3];
    wire [WIDTH-1:0] junk [0:2];
    wire carry_internal;

    genvar idx;
    generate
        for (idx = 0; idx < WIDTH; idx = idx + 1) begin : loop_main

            assign junk[0][idx] = i_alpha[idx] | i_beta[idx];
            assign junk[1][idx] = junk[0][idx] & i_gamma;
            assign junk[2][idx] = ~(junk[1][idx] ^ i_gamma);
            FA_CELL fa1 (
              .a(i_alpha[idx]), .b(i_beta[idx]), .cin(i_gamma),
              .sum(o_sigma[idx]), .cout(lvl1[3][idx])
            );
        end
    endgenerate

    assign carry_internal = lvl1[3][0];
    assign o_delta = carry_internal;

endmodule
📝 LLM result saved to llm_result_step2.txt


In [156]:
# 10. Re-synthesize modified Verilog
synth_mod_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -auto-top
proc; opt; techmap
stat
write_verilog /content/llm_fa/verilog_src/netlist_mod.v
'''
with open("/content/llm_fa/scripts/synth_mod.ys", "w") as f:
    f.write(synth_mod_script)

!./yosys -s /content/llm_fa/scripts/synth_mod.ys
print("✅ Re-synthesis complete with FA_CELL instantiated.")



 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2025  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.53+101 (git sha1 0b19f628e, g++ 11.4.0-1ubuntu1~22.04 -fPIC -O3)

-- Executing script file `/content/llm_fa/scripts/synth_mod.ys' --

1. Executing Verilog-2005 frontend: /content/llm_fa/verilog_src/fa_cell.v
Parsing Verilog input from `/content/llm_fa/verilog_src/fa_cell.v' to AST representation.
Generating RTLIL representation for module `\top'.
Generating RTLIL representation for module `\FA_CELL'.
Successfully finished Verilog frontend.

2. Executing Verilog-2005 frontend: /content/llm_fa/verilog_src/fa_example_mod.v
Parsing Verilog input from `/content/llm_fa/verilog_src/fa_examp

In [158]:
# 11. Run synthesis before and after refactoring
original_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_example_hard.v
hierarchy -auto-top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_orig.ys", "w") as f:
    f.write(original_stat_script)

mod_stat_script = '''
read_verilog /content/llm_fa/verilog_src/fa_cell.v
read_verilog /content/llm_fa/verilog_src/fa_example_mod.v
hierarchy -auto-top
proc; opt; techmap
stat
'''
with open("/content/llm_fa/scripts/stat_mod.ys", "w") as f:
    f.write(mod_stat_script)


print("\n📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'")
os.makedirs("/content/llm_fa/stats", exist_ok=True)

!./yosys -s /content/llm_fa/scripts/stat_orig.ys > /content/llm_fa/stats/stat_orig.txt
print("\n📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'")
!./yosys -s /content/llm_fa/scripts/stat_mod.ys   > /content/llm_fa/stats/stat_mod.txt



📊 Original design stats saved at '/content/llm_fa/stats/stat_orig.txt'

📊 Modified design stats with FA_CELL saved at '/content/llm_fa/stats/stat_mod.txt'


In [159]:
# 13. Compare before and after refactoring results (gate count)
def print_gate_summary(stat_path):
    important_cells = ["FA_CELL", "$_AND_", "$_OR_", "$_XOR_"]
    found = {cell: 0 for cell in important_cells}

    with open(stat_path, "r") as f:
        for line in f:
            for cell in important_cells:
                if line.strip().startswith(cell):
                    count = int(line.strip().split()[-1])
                    found[cell] = count

    for cell in important_cells:
        print(f"{cell:>8}: {found[cell]}")
print("📊 Original design summary:")
print_gate_summary("/content/llm_fa/stats/stat_orig.txt")
print("\n📊 Modified design summary")
print_gate_summary("/content/llm_fa/stats/stat_mod.txt")

📊 Original design summary:
 FA_CELL: 0
  $_AND_: 5
   $_OR_: 0
  $_XOR_: 2

📊 Modified design summary
 FA_CELL: 1
  $_AND_: 2
   $_OR_: 1
  $_XOR_: 2
